# Text Classification

In [3]:
import csv
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

import re
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation

In [7]:
import pandas as pd
csvPath = r"C:\Users\xxx\DataScience\OpinRankDatasetWithJudgments\out_sample_reviews.csv"


skewed_data = pd.read_csv('posNeg_skewedRev.csv',encoding = "ISO-8859-1")

aug_data = pd.read_csv('augRevs.csv',encoding = "ISO-8859-1")

out_of_sample = pd.read_csv(csvPath,encoding = "ISO-8859-1")


In [8]:
out_of_sample.tail(10)

,text,label
10,Could not sleep here I love design hotels so I...,negative
11,We can't recommend this hotel. We stayed for 3...,negative
12,Great Location Trendy hotel so-so staff Locati...,positive
13,Nice Hotel Great Location The hotel was a bit ...,positive
14,Great location terrific room lousy service I s...,positive
15,"Feels like Home Looks - From the outside, does...",positive
16,Unbeatable location nice clean good value. Wha...,positive
17,Location Location Location! This place was in ...,positive
18,Nice hotel great location If you're looking fo...,positive
19,Brilliant location and lovely rooms We stayed ...,positive


In [9]:

pattern = re.compile('(?i)rmb[\d]+')

def clean_text(row,pattern):
    string = row['text']
    s1 = re.sub(pattern,'money',string)
    
    non_words = list(punctuation)
    non_words.extend(map(str,range(10)))
    text = ''.join([c for c in s1 if c not in non_words])
    
    
    return text


In [11]:
aug_data['cleaned_txt'] = aug_data.apply(clean_text,axis = 1,pattern=pattern)
aug_data['labels'] = aug_data.status.map({'negative': 1,'positive':0})
#skewed_data.head()

In [12]:
X = aug_data['cleaned_txt']
y = aug_data['labels']

Feature Extraction: The models perform better with TfidfVectorizer than CountVectorizer.

In [354]:
vectorizer = TfidfVectorizer()

train_features = vectorizer.fit_transform(X)

In [355]:
#Naive Bayes Multinomial Classifier
nbm_clf = MultinomialNB()
%time nbm_clf.fit(train_features, y)
nbm_clf.class_count_

Wall time: 20 ms


array([2004., 2051.])

In [356]:
#tested KNN with k 1 through 25 and k = 20 was the optimal one.
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(train_features,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [357]:
#Logistic Regression
Logi_clf = LogisticRegression()
%time Logi_clf.fit(train_features,y)

Wall time: 301 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [358]:
estimators = [('mnb',nbm_clf),('lr',Logi_clf),('knn',knn)]

In [359]:
#Voting Classifier
voter_clf = VotingClassifier(estimators,voting='hard')
voter_clf.fit(train_features,y)

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear'...wski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform'))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

As we can see with the skewed texts(2004 positive reviews and 208 negative), the models don't perform so well.

So let's see how the algorithms will perform on the data generated through Data Augmention

In [360]:
out_of_sample['cleaned_txt'] = out_of_sample.apply(clean_text,axis = 1,pattern=pattern)
out_of_sample['to_binary'] = out_of_sample.label.map({'negative': 1,'positive':0})
#skewed_data.head()

In [364]:
#Models performance testing on out-of-sample data 
outSam_test = vectorizer.transform(out_of_sample['cleaned_txt'])
#change model name to see performance of different models
model_name = knn
pred = model_name.predict(outSam_test)
y_test = out_of_sample['to_binary']
print(f1_score(y_test,pred, average='macro')*100)
#out_of_sample[y_test<pred]

71.5099715099715
